IMPORTING IMPORTANT LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from numpy.linalg import multi_dot

MOUNTING DRIVE

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


METRICS FUNCTION

In [ ]:
def acc(h, test_output):
    TN = 0
    TP = 0
    FN = 0
    FP = 0
    for i in range(test_output.shape[0]):
        if (h[i] == test_output[i]) & (test_output[i] == 0):
            TN += 1
        elif (h[i] == test_output[i]) & (test_output[i] == 1):
            TP += 1
        elif (h[i] == 0) & (test_output[i] == 1):
            FN += 1
        elif (h[i] == 1) & (test_output[i] == 0):
            FP += 1

    SE = TP / (TP + FN)
    SP = TN / (TN + FP)
    Acc = (TP + TN) / (TP + FN + TN + FP)
    return SE, SP, Acc*100

LIKELIHOOD RATIO TEST

In [ ]:
def func(train_output,train_features,test_output,test_features):
  c0 = 0
  c1 = 0
  mean0 = 0
  mean1 = 0
  for i in range(train_output.shape[0]):
      if train_output[i] == 0:
          c0 += 1
          mean0 += train_features[i]

      else:
          c1 += 1
          mean1 += train_features[i]

  Py0 = c0 / train_output.shape[0]
  Py1 = c1 / train_output.shape[0]
  mean0 = mean0 / c0
  mean1 = mean1 / c1

  cov1 = np.cov((train_features - mean0).T)
  cov2 = np.cov((train_features - mean1).T)

  y = []
  for i in range(test_features.shape[0]):
      p1 = np.exp(-0.5 * multi_dot([(test_features[i] - mean0),
                                    np.linalg.inv(cov1),
                                    (test_features[i] - mean0).T]))
      
      cond_y1 = p1 / (np.power(2 * np.pi, 2) *
                      np.power(np.linalg.det(cov1), 0.5))
      
      p2 = np.exp(-0.5 * multi_dot([(test_features[i] - mean1),
                                    np.linalg.inv(cov2),
                                    (test_features[i] - mean1).T]))
      
      cond_y2 = p2 / (np.power(2 * np.pi, 2) *
                      np.power(np.linalg.det(cov2), 0.5))
      
      P1 = cond_y1 / cond_y2
      P2 = Py1 / Py0
      if P1 > P2:
          y.append(0)
      else:
          y.append(1)
  
  return print("(SE,SP,Accuracy) =",acc(y, test_output))

DATA PRE-PROCESSING

In [ ]:
from random import Random
data = pd.read_excel("/content/gdrive/MyDrive/NNFL_ASSIGNMENT_1/data_q4_q5.xlsx")
Random(14).shuffle(data.values)
X = data.drop("diagnosis",axis=1)
y = data["diagnosis"]
y = (y == 'M')

def five_fold(X,y):
  
  m = len(y)
  for i in range(5):
    X_test = X[int(i*m*0.2):int((i+1)*m*0.2)]
    X_train = pd.concat([X, X_test, X_test]).drop_duplicates(keep=False)
    y_test = y[int(i*m*0.2):int((i+1)*m*0.2)]
    y_train = y.drop(y_test.index)

    train_features = np.asarray(X_train)
    test_features = np.asarray(X_test)
    train_output = np.asarray(y_train)
    test_output = np.asarray(y_test)

    x = []
    mean = []
    std = []
    for i in range(train_features.shape[1]):
        x.append((train_features[:, i] - np.mean(train_features[:, i])) /
                np.std(train_features[:, i]))
        mean.append(np.mean(train_features[:, i]))
        std.append(np.std(train_features[:, i]))

    train_features = np.array(x).T

    x = []
    for i in range(test_features.shape[1]):
        x.append((test_features[:, i] - mean[i]) / std[i])

    test_features = np.array(x).T

    func(train_output,train_features,test_output,test_features)  

  return print()

ACCURACY

In [ ]:
five_fold(X,y)

SE,SP,Accuracy = (0.6911764705882353, 0.9777777777777777, 80.53097345132744)
SE,SP,Accuracy = (0.7142857142857143, 1.0, 87.71929824561403)
SE,SP,Accuracy = (0.85, 1.0, 94.73684210526315)
SE,SP,Accuracy = (0.896551724137931, 1.0, 97.36842105263158)
SE,SP,Accuracy = (0.8846153846153846, 1.0, 97.36842105263158)



OVERALL ACCURACY : 91.54%